In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [49]:
dataset_5_ms = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow.csv', index_col=[0])

In [50]:
dataset.head()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.

In [51]:
dataset.columns

Index(['id', 'time', 'activityrecognition#0', 'activityrecognition#1',
       'android.sensor.accelerometer#mean', 'android.sensor.accelerometer#min',
       'android.sensor.accelerometer#max', 'android.sensor.accelerometer#std',
       'android.sensor.game_rotation_vector#mean',
       'android.sensor.game_rotation_vector#min',
       'android.sensor.game_rotation_vector#max',
       'android.sensor.game_rotation_vector#std',
       'android.sensor.gravity#mean', 'android.sensor.gravity#min',
       'android.sensor.gravity#max', 'android.sensor.gravity#std',
       'android.sensor.gyroscope#mean', 'android.sensor.gyroscope#min',
       'android.sensor.gyroscope#max', 'android.sensor.gyroscope#std',
       'android.sensor.gyroscope_uncalibrated#mean',
       'android.sensor.gyroscope_uncalibrated#min',
       'android.sensor.gyroscope_uncalibrated#max',
       'android.sensor.gyroscope_uncalibrated#std',
       'android.sensor.light#mean', 'android.sensor.light#min',
       'android.se

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [52]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5893 entries, 0 to 5892
Data columns (total 70 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   id                                               5893 non-null   int64  
 1   time                                             5893 non-null   float64
 2   activityrecognition#0                            0 non-null      float64
 3   activityrecognition#1                            5893 non-null   float64
 4   android.sensor.accelerometer#mean                5893 non-null   float64
 5   android.sensor.accelerometer#min                 5893 non-null   float64
 6   android.sensor.accelerometer#max                 5893 non-null   float64
 7   android.sensor.accelerometer#std                 5893 non-null   float64
 8   android.sensor.game_rotation_vector#mean         5094 non-null   float64
 9   android.sensor.game_rotation_v

In [35]:
dataset.describe()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,62585.000000,62585.000000,0.0,62585.000000,62585.000000,62585.000000,62585.000000,62583.000000,54000.000000,54000.000000,54000.000000,51923.000000,57051.000000,57051.000000,57051.000000,5.023300e+04,54874.000000,54874.000000,54874.000000,54499.000000,54046.000000,54046.000000,54046.000000,53542.000000,46225.000000,46225.000000,46225.000000,12924.000000,57011.000000,57011.000000,57011.000000,50576.000000,57204.000000,57204.000000,57204.000000,54198.000000,53823.000000,53823.000000,53823.000000,50534.000000,57169.000000,57169.000000,57169.000000,54810.000000,2.037100e+04,2.037100e+04,2.037100e+04,8932.000000,12277.000000,12277.000000,12277.000000,2949.000000,54745.000000,54745.000000,54745.000000,52694.000000,15611.000000,15611.000000,15611.000000,887.000000,50927.000000,50927.000000,50927.000000,1474.000000,36665.000000,36665.000000,36665.000000,1691.000000
mean,111294.400256,1150.046097,NaN,88.238444,9.988878,9.445858,10.683369,0.861880,0.753394,0.751321,0.755145,0.004322,9.806542,9.806539,9.806545,7.133861e-05,0.325687,0.232439,0.435124,0.142522,0.324672,0.274031,0.384113,0.130444,1300.307463,1292.353364,1307.722299,40.431151,1.317694,1.202990,1.555978,0.508294,57.185426,56.938666,57.429738,0.782471,138.050851,137.859941,138.250527,1.226522,210.028928,207.937658,212.132650,5.627420,6.018476e+04,7.167994e+04,7.528767e+04,0.010155,5.363471,5.315468,5.411257,2.149151,0.764903,0.761870,0.767251,0.005218,10360.702261,10360.686631,10360.717891,0.716806,78.575667,78.575427,78.575917,0.339107,8.174253,8.174253,8.174253,0.020950
std,71786.050656,1260.411319,NaN,23.995054,1.341609,1.582800,3.271857,1.650861,0.254025,0.254559,0.254168,0.013626,0.001739,0.001746,0.001735,5.102546e-04,0.648738,0.510354,0.916648,0.332698,0.639724,0.563731,0.800216,0.303260,10149.350328,10129.382445,10182.233110,890.728298,2.169264,2.003144,3.539346,1.212473,128.059792,128.051756,128.073494,1.798792,72.106672,71.920893,72.417214,10.695213,98.649914,99.261343

In [36]:
dataset.isnull().sum()

id                                                     0
time                                                   0
activityrecognition#0                              62585
activityrecognition#1                                  0
android.sensor.accelerometer#mean                      0
android.sensor.accelerometer#min                       0
android.sensor.accelerometer#max                       0
android.sensor.accelerometer#std                       2
android.sensor.game_rotation_vector#mean            8585
android.sensor.game_rotation_vector#min             8585
android.sensor.game_rotation_vector#max             8585
android.sensor.game_rotation_vector#std            10662
android.sensor.gravity#mean                         5534
android.sensor.gravity#min                          5534
android.sensor.gravity#max                          5534
android.sensor.gravity#std                         12352
android.sensor.gyroscope#mean                       7711
android.sensor.gyroscope#min   

In [55]:
# drop std
to_drop = [c for c in dataset.columns if '#std' in c]
dataset.drop(to_drop, axis=1, inplace=True)
dataset.head()

,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000

In [54]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-54-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000

In [ ]:
dataset = dataset['accelerometer_mean',
                  'gyroscope_mean',
                  'linear_acceleration',
                  
                 ]